# Significant aspects of model construction

* econ status
* bubbles
* spatiality
* mobility

Metrics upon which to assess these:
* cases over time
* deaths over time
* effective R at different points
* constraint-specific measures (e.g. cases over time per district in models with districts)

## Undifferentiated base case

people in one big pile interacting amorphously (no status, no bubbles, no movement)


#### Cases over time


#### Deaths over time

#### Measuring R

#### Constraint-specific?

## Introducing one constraint at a time

### Social Bubbles

### Status

### Spatial Partitioning

(districts exist)

## Combining these constraints

* Social bubbles + status
* Social bubbles + space
* Status + space
* All three

## Mobility

### Weekday/weekend

### Interdisctrict

## Future work: interventions

* lockdowns
* someday, hygiene
* someday, school closures

## Template

such description wow

#### Cases over time


#### Deaths over time

#### Measuring R

#### Constraint-specific?